In [1]:
import psycopg2
from psycopg2.extras import execute_values
import faker
import random
from datetime import timedelta

In [2]:
conn = psycopg2.connect(
    host="aws-1-ap-south-1.pooler.supabase.com",
    database="postgres",
    user="postgres.rruavcjmtgpxyznzwkhw",
    password="khaibaolocnguyen",
    port=5432
)
cur = conn.cursor()

cur.execute("SELECT branch_id FROM branch")

branches = cur.fetchall()
branches = [b[0] for b in branches]
print(branches)

['015cf519-76c9-478e-9bf8-e25bb91f92bb', '8adb1159-cee3-4c59-b08f-3a9aa369d73e', '29e07c3a-47bf-4fef-8fbe-471e1587118c', 'a49d5373-fdae-4e7e-b90f-17344d0a6aaf', 'c26300f2-703c-4022-82c0-b68dc260aa4f', 'ad3cc27c-6b13-46ed-bcfd-cc099f0d5d47', '6d1f53af-555c-4296-9066-e304432d0633', '6feeeb11-f446-46da-9214-cd004151c492', '7192a064-1e01-42c7-87b9-6e6c3d381339', '0e00def4-2be9-405f-bb3e-1a2bc888828b']


In [3]:
conn.rollback()

cur.execute("""
  select employee_id
  from account
  join employee on account.account_id = employee.account_id
  where password = 'User@123'
""")

employee_ids = cur.fetchall()
employee_ids = [e[0] for e in employee_ids]
print(employee_ids)

['709dc49d-10ef-4731-a44f-728bda4ad15a', 'aa641bd4-6ff6-442c-92bd-e2f4c88e0e9b', '570bf688-fcc3-44a7-b913-d54d2c5d758b', 'f5e518e5-7491-4f59-b16c-fe4f81ce5812', '98b13248-8d60-498d-b1ea-0b3214e9c829', 'c2ee9dc7-4fa0-49c3-a714-261a71fc4412', '88501349-8466-41e4-a684-0bca467568ee', 'df0f7752-899e-4ad9-9008-712a2b9b09ea', '66d74b11-8c91-4549-8dca-cf15039c4287', '037efbc3-57e4-40d8-9a8d-00f74abd43f9', '38a1ea09-f7e7-492a-960f-a97b46d4c428', 'bd2f66be-57c2-454c-8568-38e7e38ea1db', 'e8280f8d-ca5f-49ed-ac24-f064999e8157', 'c7abbe4b-deac-4853-8275-4703c6fcd6dc', '9a56725a-1465-4b13-a3fa-77985fdd49bf', 'c07511c4-02c8-4375-899f-b0fd4f47ebd0', '486ce97d-df5e-4221-bf5c-3bd59764cfd0', '5ee6e78b-2f3f-4bfa-bcda-8c70cc8344b0', '02b5691e-d603-4621-b673-7b87eb9c35fe', 'cb3f5a07-33e2-492c-87b4-1d6e6fd3475c', '55e0304e-1c28-45c2-b4af-9db33ad99df3', '0f12fdf0-0d4e-4638-bba4-8e70160bd3b0', 'f6b22555-1266-4526-9efb-eb64d4da4b17', 'd231da32-d755-4978-a029-f25f9413872a', '0c54eb2a-3c67-4321-9cc6-7a1a77743731',

In [4]:
history = []

for i in range(300):
  branch_id = random.choice(branches)
  employee_id = employee_ids[i]
  start_date = faker.Faker("vi_VN").date_between(start_date='today', end_date='+30d')
  position = random.choice(['staff', 'manager', 'supervisor'])
  salary = random.randint(5000000, 20000000)

  history.append({
    'branch_id': branch_id,
    'employee_id': employee_id,
    'start_date': start_date,
    'position': position,
    'salary': salary
  })
print(len(history))
  

300


In [5]:
query = """
INSERT INTO employeehistory (employee_id, branch_id, start_date, position, salary)
VALUES %s
"""

values = [
    (a["employee_id"], a["branch_id"], a["start_date"], a["position"], a["salary"])
    for a in history
]

execute_values(cur, query, values, page_size=len(values))

conn.commit()